In [43]:
import pandas as pd
import numpy as np
import sklearn

from tardis.io.config_reader import Configuration

ModuleNotFoundError: No module named 'sklearn'

In [38]:
class tardisGrid:
    
    def __init__(self, config, axesdict):
        try:
            tardis_config = Configuration.from_yaml(config)
        except TypeError:
            tardis_config = Configuration.from_config_dict(config)

        self.config = tardis_config
        self.axesdict = axesdict
        
        return
    
    def getconfigitem(self, key):
        keyitems = key.split('.')
        val = self.config[keyitems[0]]
        for k in keyitems[1:]:
            val = val[k]
        return val

In [39]:
axesdict = {'model.structure.velocity.start':np.arange(10000,15000,1000), 'model.abundances.He':np.arange(0,1,10)}

In [40]:
grid = tardisGrid('test.yml', axesdict)

In [41]:
grid.getconfigitem('model.structure.velocity.start')

<Quantity 10000. km / s>

In [45]:
from sklearn.model_selection import ParameterGrid

ModuleNotFoundError: No module named 'sklearn'